<a href="https://colab.research.google.com/github/jesung/Tic-Tac-Toe/blob/master/tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pprint
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random

model = keras.models.Sequential([
    keras.layers.Dense(32, activation=tf.nn.relu, input_shape=(1,9)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])

class tic_tac_toe:
    def __init__(self):
        self.board = np.array([[0,0,0],[0,0,0],[0,0,0]])
        self.turn = 1

    def reset(self):
        self.board = np.array([[0,0,0],[0,0,0],[0,0,0]])
        self.turn = 1

    def move(self,x,y):
        ##### need to update #######
        if self.board[x,y] != 0:   
            print('Invalid move',x,y)
            print(self.board)
        else:
            self.board[x,y] = self.turn
            self.turn = -1 * self.turn

def win(board):
    for i in range(3):
        if board[0,i]==board[1,i]==board[2,i]!=0:
            return 'Won'
        if board[i,0]==board[i,1]==board[i,2]!=0:
            return 'Won'

    if board[0,0]==board[1,1]==board[2,2]!=0:
        return 'Won'
    if board[0,2]==board[1,1]==board[2,0]!=0:
        return 'Won'

    if not 0 in board:
        return 'Drawn'
    else:
        return 'In-Progress'
          
def move_gen(board_state, turn):
    legal_moves = {}
    
    for i in range(3):
        for j in range(3):
            if board_state[i,j] == 0:
                tmp_board = board_state.copy()
                tmp_board[i,j] = turn
                legal_moves[(i,j)]=tmp_board.flatten()

    return legal_moves
  
def move_sel(move_list, turn, train=True):
    tmp = np.zeros((9,12))
    i = 0

    for key, value in move_list.items():
      tmp[i,0:9] = value.reshape(1,9)
      tmp[i,9] = model.predict(value.reshape(1,1,9))[0][0][0]
      tmp[i,10:12] = [key[0], key[1]]
      #print(tmp[j,9], (tmp[j,10],tmp[j,11]))
      #if win(value.reshape(3,3)) == 'Win':
      #    return tmp[i,0:9], tmp[i,9], (tmp[i,10],tmp[i,11])
      i = i + 1
    
    if random.randint(1,5) == train:
      #if turn == 0:
        #j = random.choices(range(i),tmp[:i,9]**1)
      #else:  
        #j = random.choices(range(i),(1-tmp[:i,9])**1)
      j = random.randint(0,i-1)
    else:
      if turn == 1:
        j = np.argmax(tmp[:i,9])
      else:
        j = np.argmin(tmp[:i,9])
    
    return tmp[j,0:9], tmp[j,9], (tmp[j,10],tmp[j,11])

def simulate(batch_size, num_round):
  for i in range(num_round):
    board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
    train = [np.append(board,prob)]
    
    for j in range(batch_size):
      #clean up - train & tmp outside for loops and clear at beginning
      board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
      tmp = [np.append(board,prob)]

      while win(game.board) == 'In-Progress':
        board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
        tmp = np.append(tmp, [np.append(board,prob)], axis = 0)
        
        #print(move[0], move[1])
        game.move(int(move[0]), int(move[1]))

      #update score based on result
      if win(game.board) == 'Won':
        tmp[:,-1] = (1-game.turn)/2
      else:
        tmp[:,-1] = 0.5
      
      #add to training set
      train = np.append(train, tmp, axis = 0)
      
      game.reset()
    
    #print(train.shape)
    #print(train)
    print('Round', i)
    model.fit(train[:,0:9].reshape(-1,1,9), train[:,9].reshape(-1,1,1),epochs=3,verbose=0)

In [0]:
game = tic_tac_toe()

simulate(500,400)



Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [72]:
#game.reset()
#game.move(0,1)
print(win(game.board))
print(game.board)
print(game.turn)
print(move_sel(move_gen(game.board, game.turn), game.turn, train=False))


Won
[[ 1  1  1]
 [-1  1 -1]
 [ 0  0 -1]]
-1
(array([ 1.,  1.,  1., -1.,  1., -1., -1.,  0., -1.]), 1.349234864153994e-15, (2.0, 0.0))


In [0]:
game.reset()
board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
tmp = [np.append(board,prob)]

while win(game.board) == 'In-Progress':
  board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn, train=False)
  tmp = np.append(tmp, [np.append(board,prob)], axis = 0)

  #print(move[0], move[1])
  game.move(int(move[0]), int(move[1]))



In [70]:
print(tmp)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  6.38136869e-14]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  6.12533158e-12]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  6.64533871e-14]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.28533021e-13]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.00000000e+00
   1.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.42437528e-15]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 -1.00000000e+00
   1.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  3.38241273e-16

In [0]:
def move_sel(move_list, turn, train=True):
    tmp = np.zeros((9,12))
    i = 0
    if train:
      k = 0
    else:
      k = 10

    for key, value in move_list.items():
      tmp[i,0:9] = value.reshape(1,9)
      tmp[i,9] = model.predict(value.reshape(1,1,9))[0][0][0]
      tmp[i,10:12] = [key[0], key[1]]
      #print(tmp[j,9], (tmp[j,10],tmp[j,11]))
      #if win(value.reshape(3,3)) == 'Win':
      #    return tmp[i,0:9], tmp[i,9], (tmp[i,10],tmp[i,11])
        
      i = i + 1
    
    if random.randint(0,3) == k:
      if turn == 0:
        j = random.choices(range(i),tmp[:i,9]**1)
      else:  
        j = random.choices(range(i),(1-tmp[:i,9])**1)
    else:
      if turn == 0:
        j = np.argmax(tmp[:i,9])
      else:
        j = np.argmin(tmp[:i,9])
    
    return tmp[j,0:9], tmp[j,9], (tmp[j,10],tmp[j,11])